# Modified Nodal Analysis: Project Report
Author: Philippe Nadon

Date: Nov 20, 2019

## Introduction
Oftentimes in electrical engineering, the voltage at each node in a circuit needs to be known to understand how the circuit works. As the complexity of the circuit grows, it becomes increasingly diﬃcult or even impossible to use many of the common ways of solving the circuit, let alone doing it by hand.



## Introduction to Tool
This project explores methods in which one can implement a circuit analysis tool, using [Modified Nodal Analysis](https://en.wikipedia.org/wiki/Modified_nodal_analysis). A large portion of the work in this project is thanks to [this tutorial](https://lpsa.swarthmore.edu/Systems/Electrical/mna/MNA1.html), which was immensely useful in developing this software.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16,10]

In [9]:
df = pd.read_csv("stats.csv")
df

,fname,matrix_size,num_components,processor,total,transfer,solve
0,rand_data/rand_size-003_density-002.csv,1,3,cuda,0.539095,0.253007,0.286088
1,rand_data/rand_size-003_density-002.csv,1,3,cpu,0.000956,0.000261,0.000695
2,rand_data/rand_size-004_density-002.csv,3,6,cuda,0.169986,0.000830,0.169155
3,rand_data/rand_size-004_density-002.csv,3,6,cpu,0.000709,0.000267,0.000442
4,rand_data/rand_size-005_density-002.csv,7,8,cuda,0.002514,0.000718,0.001796
...,...,...,...,...,...,...,...
215,rand_data/rand_size-022_density-010.csv,7169,20476,cpu,3.563336,0.013422,3.549914
216,rand_data/rand_size-023_density-010.csv,10098,28957,cuda,0.011335,0.000000,0.011335
217,rand_data/rand_size-023_density-010.csv,10098,28957,cpu,9.680842,0.018369,9.662474
218,rand_data/rand_size-024_density-010.csv,14311,40956,cuda,0.015637,0.000000,0.015637


In [11]:
sns.scatterplot(x="matrix_size", y="solve", hue="processor", style="num_components", data=df)

ValueError: These `style` levels are missing markers: {509, 764, 267, 1806, 19, 1276, 23, 536, 28, 29, 30, 28957, 1533, 3617, 38, 40, 45, 48, 2045, 10237, 1081, 314, 60, 61, 11580, 11581, 2557, 7236, 3068, 2892, 2893, 79, 83, 86, 87, 93, 356, 359, 106, 4092, 20476, 4093, 12285, 2168, 123, 636, 124, 125, 23164, 128, 381, 896, 5116, 14477, 155, 5788, 5789, 1444, 172, 176, 6141, 188, 446, 717, 720, 215, 17372, 8188, 40956, 8684, 508, 16380, 8189, 24573, 32764, 4340, 252, 253, 2044, 1021}